In [303]:
from langchain_groq import ChatGroq

llm_model = ChatGroq(
    temperature=0,
    model_name="llama-3.1-8b-instant",
    api_key="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"
)

In [304]:
from langchain_community.utilities.sql_database import SQLDatabase

uri = 'mssql+pyodbc://localhost/TPMS?driver=ODBC+Driver+18+for+SQL+Server&TrustServerCertificate=yes&Encrypt=no'
db = SQLDatabase.from_uri(uri)

db_info = db.get_table_info()

In [305]:
query = "Give me ADEO project list"

In [299]:
from typing import List
from pydantic import BaseModel, Field

class SelectedTables(BaseModel):
    """List of selected tables"""
    tables: List[str] = Field(description="List of table name")
    reasoning: str = Field(description="Reason for choosing the table")

prompt = f"""   
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>  
You are an Microsoft SQL Server expert with extensive knowledge of database schemas, relationships, and querying best practices. 
Your task is to determine the appropriate tables and fields needed to answer user queries related to a given database.

Note:
If it is only project related details then use only "pc_work_group" table
If the question is about milestone then include "pr_ADEO_TaskMana_Data_Mileston" & "pc_work_group" tables
If the question is about delivery then include "pr_ADEO_TaskMana_Data_Delivera" & "pc_work_group" tables
if the question is about milestone and delivery then include "pr_ADEO_TaskMana_Data_Delivera", pr_ADEO_TaskMana_Data_Delivera, "pc_work_group" tables

## Guidelines:
1. Analyze the Question: Understand the main entities (tables) and attributes (fields) mentioned in the users query.
2. Identify Relevant Tables: Based on the entities in the question, choose the tables that most likely hold the data needed to answer it. 
   Consider table relationships (e.g., joins, foreign keys) and context to make your selection.
3. If you are not able to find the most relevent tables for users question then include all the available tables. 
5. Do not create any select query in your response.
6. Do not give any additional information
7. Sample json response format:
{{
    "tables": ["table1"],
    "reasoning": Reason for choosing the table
}}

Users Query: {query}
Database table schema: {db_info}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

stuct_llm = llm_model.with_structured_output(SelectedTables, method="json_mode")

selection = stuct_llm.invoke(prompt)
print(selection)


tables=['pc_work_group'] reasoning='The question is asking for a list of ADEO projects, which is related to project details. Therefore, the pc_work_group table is the most relevant table to answer this query.'


In [300]:
selected_db_info = db.get_table_info(table_names=selection.tables)

In [301]:
class SQLQuery(BaseModel):
    """Response SQL Query"""
    query: str = Field(description="SQL query")

prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an expert in Microsoft SQL Server with extensive knowledge of querying and SQL syntax. 
Your task is to generate the most appropriate SQL query to answer the user's question based on the provided selected tables and database schema.

## Guidelines:
1. Analyze the selected tables, relationships, and fields to construct a valid SQL query.
2. Use the relevant tables and include all the necessary fields in the select query to give detailed answer to the users question.
3. Ensure the query is optimized and includes necessary joins, conditions, and aggregations as needed.
4. The SQL query should be based on the user's question and the schema.
5. The response must contain only SQL query and end with semi-colon(;).
6. This SQL query response will be converted into natural response to the user, so include all the necessary columns for the table to given more information.
7. Sample json response format:
{{
    "query": "select col1, col2 from table;"
}}

User's question: {query}
Selected Tables: {selection.tables}
Database Table Schema: {selected_db_info}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

stuct_llm = llm_model.with_structured_output(SQLQuery, method="json_mode")

sql_query = stuct_llm.invoke(prompt)
print(sql_query)


query="SELECT pyID, pyLabel, ProjectType, CompletionRate, PlannedBudget, ActualBudget, StartDateTime, EndDateTime, DGTitle, EDTitle, Sector, SGTitle, UserDepartment, Classification FROM pc_work_group WHERE Classification = 'ADEO';"


In [302]:
query_response = db.run(sql_query.query, include_columns=True)

In [192]:
query_response

"[{'pzInsKey': 'PC001', 'ProjectType': 'Infrastructure', 'PlannedBudget': Decimal('1500000.00'), 'ActualBudget': Decimal('1450000.00'), 'Name': 'Initiation Phase', 'StartDateTime': datetime.datetime(2023, 1, 15, 9, 0), 'EndDateTime': datetime.datetime(2023, 2, 15, 17, 0), 'ProgressRate': Decimal('100.00'), 'Status': 'Completed', 'ActualCompletionDateTime': datetime.datetime(2023, 2, 15, 17, 0), 'DaysTaken': Decimal('31.00')}, {'pzInsKey': 'PC002', 'ProjectType': 'Healthcare', 'PlannedBudget': Decimal('2000000.00'), 'ActualBudget': Decimal('1950000.00'), 'Name': 'Implementation Phase', 'StartDateTime': datetime.datetime(2023, 2, 1, 10, 0), 'EndDateTime': datetime.datetime(2023, 8, 1, 18, 0), 'ProgressRate': Decimal('100.00'), 'Status': 'Completed', 'ActualCompletionDateTime': datetime.datetime(2023, 8, 1, 18, 0), 'DaysTaken': Decimal('182.00')}, {'pzInsKey': 'PC003', 'ProjectType': 'IT', 'PlannedBudget': Decimal('1000000.00'), 'ActualBudget': Decimal('450000.00'), 'Name': 'Planning Phas

In [193]:
response_prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are AI expert tasked for create a natural response message from the SQL query response for answering the users question.

## Guidelines:
1. The response must be concise and directly start from the response.
2. Tone: Keep the tone friendly and helpful, even when there are no results or an error.
3. Simplicity: Avoid overloading the user with technical terms like "SQL" or "query" unless necessary.
4. Clarification & Suggestions: When no results are found or an error occurs, gently suggest possible next steps (e.g., adjusting the filters, checking the database connection).
5. Summarization: Present the data in summary form when possible to make it easier for users to grasp the key insights from the query.
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
The user question is: {query}
The SQL query response is: {query_response}
"""

In [194]:
message = llm_model.invoke(response_prompt).content
print(message)

Here are the project-wise budget details:

**Project Budget Summary:**

- **Infrastructure:** $1,500,000 (planned) vs $1,450,000 (actual)
- **Healthcare:** $2,000,000 (planned) vs $1,950,000 (actual)
- **IT:** $1,000,000 (planned) vs $450,000 (actual)
- **Education:** $500,000 (planned) vs $100,000 (actual)
- **Energy:** $2,500,000 (planned) vs $1,500,000 (actual)

**Project Status:**

- **Completed:** Infrastructure, Healthcare, Energy
- **In Progress:** IT
- **Cancelled:** Education

Let me know if you'd like to filter the results by project type or status.


In [256]:
import base64
import re

def convert_file_to_base64(file_path):
    try:
        with open(file_path, 'rb') as file:
            file_data = file.read()
        
        base64_encoded = base64.b64encode(file_data).decode('utf-8')
        return base64_encoded
    
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' does not exist.")
        return None
    
base_64_string = convert_file_to_base64("transport.pdf")

In [257]:
print(base_64_string)

...


In [5]:
base_64_string = """..."""

In [212]:
base_64_string = "iVBORw0KGgoAAAANSUhEUgAAAUYAAACbCAMAAAAp3sKHAAABlVBMVEX////GAAvXAA/sbAC1AAXFAACQAACxAAC0AADsaQCwAADTUVX97+PCAADtbgDVAA7KAAu8AAfrZADrXgDQAA25AAb/+fnKJir00dPIICMAAACLAADBAAnVV1vkmpzqrbDwyMrRP0XviT/mpKXyoW3LMzXgjI740brwjEz89PTUZWb0qnz+9u72wJ/fAA/ovL3LNTkjGBXufTX35eXufSfwlFb3yKnmzc3KZWb56uvGVlfy2Nl6AADTAACcJia6Gx3UgYK4EBO9NjfCQUPksrPQdHXacnXSPSzeXzjVQB7j5eXlnaDqaxbQLRTnbDHoaSLZQADcVCr1to6Xk5K9u7tKAADULQCoCxF9EBI+AABtERNZExNGFBTQpaXad3q6JyjkdE70wrPupYnsfEDeXETdUhzWQBLbamDROCTVT0RfOTloUlLdYDjvfS5nZmXJzcx1fXxGLSxHPz5OVlSgpaQ5FBPys6AADQjaY1wYAAAWGBUkAAArFhQwNjM9FRRVEhO8f3+oRUWtX1+5cnLIkpKcJCSlPj42HSBHAAAYcUlEQVR4nO2diX/TxrbHLVtCYyWS5ZUsSuI2EHAAuyGBxjE0zgZNcFYKpGEJt5BcetsUyvL64N3XC31t/+53Zh/JspNQWpNUv8+nJRnLsvT1mXPOnJlRYrFI7TXf6Qs4HqoPdvoKjoVqVk+nL+E4qIqmC52+huOgWbPe6Us4DppCZrXT13AMtIDMkU5fwzFQ3USznb6GIy7cmwGjOd7pCznSKtbhfwumZ0U5+B9QkaSMU6ZrRin4+6tiIfzPLMpFKc/7q2iiSfinqqO4Ve/0xRxZ9VjInCf/piOM761xBMIBum5qcSvyje+paaTTdHHbzOWsSqcv54hqwdR1tBDDfVqPu9Fo8P0E4SWfN7ENTppe3J7r9PUcUW0jPadbVeIiUynzRqev52hq3tLdHMJZIw4wnhUVHN9L2ygf90zIGgsIxXNRuvN+Kpq6G8/jEWDFcuMIlTp9QUdTk2CMKR1HmLqVc62o2riPaqF2Bj3Zi+d0XB2bNFNeVCXbTwU0EhI9ipaei7sIQaAesXI2+uuv66ipaE03lxIHTT0V19B0CYdsNx2VvvfXpGlNBYcoXeAaAeM2GGrJ9KLc+wCq4gpEwCBnwDXGPYIRog3a7siFHTENmzoyR1UPCRFGw9Y4g+NPzUSRbzyIYNynm9M12TBOMNIQg2sUZq3VWyNJwcBP1xGSC3VqpunGccJDqmOFGWuoY9d2lATmmIaOXeS/U4wpVm6EX6c6dWVHSvOmnnYVjoAROnVc09l8YNGS+Xc1Ghe2FK6KYY6sX4/jQQwor5s0hBf5FELPVJT8tBYE63wcOCJmdQilMcacaKFxvGfKtKJxYWsVTDBH6MR8IeMc0jFGTHZbZELVKQtF09VtNYl0nHDruMIYI9bpEo4ab4nFhhDSo4pZe/XgXh2Pp3W6VqdqUudIWkjWU52zcHY53NnL/NhVgCCTIjmOSextivwaJy0z0K0rYIpaHs10+jo/di0g0o3BGZJODCmQFmfu0bwRq4Mp5nI8bkdqqYqpe7QTk6ynoOMaD1EebS9Yej4VTyM9mthqqSrxd1XIwFmOQzLDERNCN4/W4BVTeGwYLUBpo6lZ7A4Ry3E83cKFiHFLZ0EmBQNujNRjlYpI4SpaswUZVHLMO4ogA2MZYpisPVILlWbMUdKLKbY0tbp5njqCfdLgYxU7faUft7oQDPGGuTN0aUCusgEhYPQY3U5f50euGyYEjyGOETJFUhebYs4yrrHWhU5f50euCl6bPC9Cs4dM3CrMk7lMM1rh2F5kiXdFyXCIFywK50hMMorTbVUajvVYZl3BmKIJYs3iA5l4DrvGaLK6jcbRIGQ8Vo/SqeM0Ay8JjCktSnfaq6qjQqxizSquEFseLjsWRALuudhCo+JOSxW2rRsQqfG4RcEIfhAPZDjGnJWKIkxbLZi4DjuL80SRfpPMsUg6NcWYzpOmCGMrkdgSGyflr1GeJpLkZoiMqjX6mxdhbKtZZIksZltUxrAjrJPFeSThSWOaWoSxpSDtlvP4YtoAMObxiIWF7hQpTEQYW2sOyfBbE+VuXNPB2Q1zlhquNeJO/XdeuDw42vq1qkVLi0RK2ogznikxyZrXdeIgUbsFKMXJYz3gHrLtgZYvwrhFLuqelBEGY5zDFR6P9uk0TclbPouncANZc8d5eqFmuJodrPxP8Q1XdQVjYYaXxThGVm90dZr2eLpcKeVXz7RpHWtbLBmGpml2YDqvNGtRjziJ8qJTD/vqELhTsz7tMYwptVer6/QGLXTMBzhdmKLmGoFFnoUpaxs3TaEcA4qX1yp9GkJMvcfiM4UsCfcURzo9LM9lomM+7XrN1oiM3uCKkUkT59ejKM7rDZNINUZcop2lUzF4PoslQmzKEKvC5xMKs0DxeGdCw4wicPwk8FIB8u5KbAHFXZp+D7PZVVISI+zwIgk6hNH5S64u10Fts+LjlHncSxY9hiZkPwi8iJ9/APBSceLwKhafBczlKDABD//IRzdpOVE9aBFHOWzpx7wMWTUUjJod3A5dB49Ws9x4zsLTB4jnjBqhqWF2bJoQE03xvq7znazjJu7WJbDZVvH7mOgzlSJwLPpfBnQL0DXB3KxZJBxjirrBvEBHpvpFXg493JylvXkamzHeCfJnbJkpKPoTTh/+SaGvj/opaobhXyUL48DBWI+F9xsgpdBIf5AG6BGMymBbp6VIvOeoEJtB6nTh+EAX1zV6UTXZ8iDYoCiQSJS6ek8qKor20bA3g0JHqYVr4vUBVn7pkW9hzmlA+aRPugZGKoFYPGhrARkn1dfBGPEzOAbNHJ4iEKPAPDe6FI8qVLwQibnSjR+Acbxm6eoas1Hb4LJp2BlQWkg475INij7zX3s9cJC8p7A343OH7doZUj6bDYg/Cb6lFjyjbfeOKCcrNlEEjsqosJRH1IzmkCQE1FiGmJfOkEoZ4Lg6suZJiBqHcY5a9xmQHcCg33aXbLGH6I00X5dmnPYDCLoj3o1Gwt4c5rCIhpWr6aJNJ43AW3qaMQHKAb4/aNwI+0RbGv+gqZNHuJW2UV7hmKfpTpo35XXBMaVyNG9M2xDp6x8Io98aq4FjxCCsNcawBxirGJkBnTYCbwnBiA+3u0jfLvSGf6DNx3AFlM7jHQVAUfo9AESNLsVtUVJUwgzdnmAYYJILSFd94wfCGOxJBk/W/iqMeLyCe8CojeWzSIM08f5RsVzy3IMpU+2ucQWVYov5NP1XjnJcLZ5CFRKoPbo780NiDNISY4e/DiMNyMV5IsUVGCMVIhbV6xBZ0tZI3RSZNaGTb6aYd7Fp5jxfvAal8bgQht25vCnKFB8IY1cQo1b6yzGCvxYJ0OmgRxWqmy6kiJaphmAwRlelmPZZYC4tM/K4GBdaekqTo5gPg7HQdAgPxH8lRs2+tj/GouVRNGov1lRjJH5RUxriubziIOFlPIoxTXCUFu/VHwZjc5LBsqc/HWOLBKE1xticSWOyCs7HFJjl/Z6SpuLcPCHMTMdqFpDXxYP+D4sxPG8cbILFY8yfi9E1NH804ebYBiN+7FgAUloJNtgWvXiTXIWjppsLs3i3cF48kfCQGI2uUaop38XJQ/gPfOBQpJHTR5g2hS11C8F4sh1GCCo1dehn9Bb2xRjrmQlwdFU3mdf9flLF64pjTBOjfl+MRrDoRFXg9NyTIg232Q2N94Dq6q2eLuKm0P13IRgfe60xGr24QeXIXXI7jMoyPNal1SkEPR+0VaVju7xbp8kJZE38cBjt8AcGiPIeGKsRevVKUmy3KRiHYFy/KDiGYMTf1bhinvwC22KcQ6r5pfLqfJYV0qG5NBFnWIFS7n87CEa76SoDuiEwjjwQP/qqfL3tbkvBqCBiGL9MfCm+mXCMMSXOGCP7Y6wpaySI/SnFB8sXoYNydT9/6zB548Od9WVmETv/CL17YYLG/Ig0zPfA+M3azjI96vYjhjGTmOjdB6N6D/X9MS74jNFTOrimByN0E0dpuTlkhQ8GHzTdMcH4OOE4q4TjTiJ0Hky4RrhkaZi+4tQBMV5IOs46OcrZYRizCef6PhivhWBcXrRbfF4JqbPSmmJ/brqVW5SHWNw9ppH6/GUFo2aMXhu9psZl1ql3nYSzCr8uO84/S4VSqVQ477uwmrgxu1aRc0lqrfSAGOeziYTzLZwi66xQjOeh5bHRHuNnYRgnnrT4PN9KE001r9AI3WSP5J+U7l9VDxg9eedYkqLwjY2E0wBz3HGcXaMXtDcxcUY5h/RoNr/J3XV/KDkgxorhJBLfwymSCWf0oBjVECN8423WgZo/b0qZlc6pzm5fU6Qc2fJROYIhGJd3Vld6tXBxjCuJBGD0VsFUVr7VFhuJZOaScg7pGrVYDf/sNZxeYRgCo/dkp7Gj3FZJlYJxHdt9InOOY2Sd2muR8FRPq/2HDZ5OrznOYjhGpCzCa5EjEpXLV69eLZebOaaoVfqmqB86jvNYtcAQjBrcWwPuYy8BguPh/0kVo6GElarhasuNxEogxgBGbx3euixvq8dWRbtwZc0hxufDuHcSn3v5S6M5/bZr3ZcXDfYVSoOFfp5xEquhGHuUlSZpX63Mx9C4s7n1r+/+tdVvN4FM0W/AtzthIpFYkQluOEYwRgfMSFvFAKlUjDU5hKkTYA3aMQ3lY6B1HTzsjidv64H65bl0UPOPTIIEZh9Gmjiu7fU2Y3wC/YJRfLhIWDKHdZ2582aMw+oy7xaRuYxu/fDVEjn86Q9bV1lroCLpm6QGjA+bjHFn9/ayxPgQ2yB22DvhGIekb72B/f2iQw9XZ1t6jV74JNLPfBi99duLHsZEMZ4DjI/wr0lHYnxEzo9DTxNG7GaYETxxdmUPKMALLaxxUq5PzocbY9m49WxJvuHpj8wgU77M3L/DWmJc3Flb519yw3Eauxzjp0niGomNhGGU+Qa+4lEb39uy+BI4xscJ3uyzxkXwEavLKkZyOWtOwBrhhcdNGL0JdmXkRAn5kefxla6EYpwV6Y4b7hnL1vOnvncUnhvsDSp1TVkdRTCSr38ZyAE7DnIVfslKjKRPq+aoYjxtLHJrHMdTgQ1mvKLcIjGu4U7NUvgRisPDh+9KjBPfErxqiFlZ3Engzt4Oo/E9nEVMSGKMDs14VvwYS+SxgswY9ZDgXL5yMxbUFvOPaiXI/wwP3NVWer1dCognv/TmdiRG1twIwVg1Gqz7kNVbQ9gaE1/a/gS81zjpEI6ulr1Am87dpu/CNg4EOMbGSRe37iYFRnxZ8N+XdihG1qm9i3ByMZGGMTae0LPv+jBWTVlfCCuIlbe+aqIYu2nTF1OWq2JU0u8JQm+CUGys7uzuegpHQ2Bk9ibMUcE4n+GvkgmYoo2/ksZ1j3dUhlF7yOJ8hmH8Zpe7EPClSYmRZV9ZBSMWBEL/BCt8qEdcIL5i7xGc+7aoPxGMJ8ltOGs+jDVL7tsP6dNXb/k7NNVSPzNHzxL2m9LVZ21dFO4OXDKeSOff9Tq2nUCn1q47SSolbzyXIS5vEY+C8Ndta4TX6u5utlvBaJ+8mCQfwzE+4J9EvrHbAuP1UIyPgC5d8tlj4CPsVdwF1rGZri4uLmKea3LFPLVGmyQXfoxFMYbJ5xHkhQehGFu6UuYmmJYY1a3qA0luYKs2CXPjhuxrWRFi9qiRrifPXqKShJwkwdhYZLUNyPqS1PAylxWMmvHZng/jDb76wV5LZjJrBYaR+w8fRvA7wORqN8UI9mevOySCrDs8lU1mlYhGQkxjfb0Bhu7v1KLWmENXnj97sWWoIK9+LSL0VzeVYC0wgj+Vi1XUxHHgNsMIHtxX4cEDaYnRWfeICZxr+qa6s6TLe9lVNvzDN0zPmZFH41GM/dCHMXZjhOmbjY0N+rXgSO2QctIy840F4htparvGMMLngQUTh+wRjtA7smfVITzBSFwqWEc4RvfWSwzouVUOo3izr++15PiUd2rwBJ7EqOzyGLBZcHls+Atl6/iOBUZsD7293yv2xdWTwUMcb9mZ8OxLZ7preFrGywYdKMF4+qIPY4gwRvAgy0BRxcgGCGukItKz66zby2yQYrjJbCaxd+lCt+88551kQliHDyPv1OUrL2nDM4FIobjUd1fr+0G86SaSgYUPyFN+a7Rp9IVe68NIHJa0RhKHJhIhGOsw7HIaWezqPSeb2cUez6OhyNHE6P0wGHEndZzkKQoEzszr3wbB2J0EawSMDvaWA5UzZ7q7m0+0kaXKGLZ/0TKepifMXvCWZ6zHXr0nze9m353yqz5x9feExaZkATzv840GtcYV24eRZkA+jARBM8Y5nivt0DQO0gxvmZ9TJKgYo3GdNGf+qzXGjYz4qORZ0nImmxBjfhrNu7MUIw7R9qGfqDhOI3X5iowlNzHHsvFKcYYvfpop3+3jqc/LE7LjixlsCDF1+Yb/ztBb3vFVv1keuUMwPmqHsWrgERnGCKFhhYTo1SwHa4sV5rg08YQ2J9tgPCMxJhqk5QIk5Cx2s0UR4Ix7abIJCVrbBannQyYfq8gkGG/JpqW+/nLZ/lGhGBsDjFt9LA9fuqWEIVHN8OeNPKGWGM/edpdZp7xNSxMKxqYOCVlcLz14GedDCUWAUYxjHq6tMbgHxpjZgIbCXlJipAkNWKPHM1inDcYLF1e13gdBcy2RR/XHy1/LpuebP1nWC9+BfW/ulO/1PSM/F340JEVtM58XGJVRDL9xBSOrhUmMO4qJXD7DxN6PI8oqx2ivT6gYV+1e/jGnRF7VtlMXFIyJ7KULGxM4S2AY2YC5O4uzL4+Um1pjPH8x6yzjdKoYeGGOjKnLW8L4nr6Ov3keGAD2vdksv3lNMC79aEuK5a3NPBtBur4xNb+7PeEbzwrjC8GYyDBlaQQg1XOBUbN7lSoQpGw271an2lm0oomk8lmQTpJfv2e1xHGGkQ6jnzTaYbyYYaE8uE2LVnjK/cI3vvw61RdMusfG/r315jVmu/SzYotxbSzNB+L+Cg+/a/jKmzCyEPOJemviFi/S9xs0qGN7Jjdnr6r2uMZXmBwU40Ym0aw9Mj7kRTDo1E/46KclxstZnscbgV2srN5oP+MNN/8n1RccR794PfZmDLe+vHU1rhpjv6hn+OuNwnhW7GaMGVKMeRhyZxxjzcYDCmZ75LIbvqM2DomxO5sIEckbef4HhzRwho5DVkuMZ2U5JRDNWfX76s+8+WXfZpM13uwbGxuDhOeHft9o0ft3jq+pD1S/BUbnut2MkXjAT9tY45DtLcrD19bWeEdkR/EE/KAYY5fCPsxZ7fVsPieBSTvZnR3sPlpihMiUYHWA4JorNhfTz83xad+bvqXA25cwxh+++jowg/BqRmAMzMVIV+bsDB8e4z+zGVH0mWDBR/FvyYnCITGGmyOAvF0LOaIdRqcRjpHtQUe8ClF6/fr5Uog5vvnffh/DeHlzq+zx7W+BmUElIrCk8FAYJxKO/+DAffIiz4ExhntHOJHIWA+EEd8Ec6HBFJ1tOc+jrxm7l2Nf3dpq4vjTlus3xfKdV6TUm2Z92rdf8I9h7FbuOSvOqSZ/Zw6LMXY2jGNWVjkOhPEyPohE9GCI4asmPB3dYsPqwgu7fM/H8emzrXSghlbOv/HInlaP9WnfApI/hvGyMnhzxDn3lDNsHBpj7FRzv1YoHgwjzZxgZGobTX8OYsGkjzPR01e+w/BIlefq5jMOcunlj1eaplXL2thMWS66zQee7K9gzDZhzLbBuEfuWGJUCrlKa3KiqWlfjDAAzPjTx6Q6BlUwJhMtMXZP4HM4mZ1i00t0RRnehZW3rtz97u4VUnMs2/Dzs2fP4HcUrObCq97YnTLZUEhKE7ng40U3Mkkup5vdhGhpnA80SFEzU96dkHWW7oQ87FPadKbpY9rp/LlGhr8hk3E2/EuGLsqzn2pzkgsT8Fnnzoe8Qtc3ku0G6bJcGEGWSYQulIiXZ96ALdJthNQYg39hq1uIf7PnRUtTg6Lgu9XrVd5QaPkx7dV9+dSlTycmPr20cab5teAFttD5Fq8XyaMkNLJxqO16RkHxBKEY5ztk3PYP5vm7qItsRQ9uZGtNcevnHKao8eObjfFvKfCOab5fOr8vxJl/b5IfcvxwvBOh07fwUWgB54453b/vPBxifHNshnrLPNu7lYv+AA9TQcfd2tufY/kOmCKlSHdhEpzW8X74zsFVwTsv+c7zlhzLM/fuenzhCd/K6unRXx4UquN9wFo7jmUMkY9lUmm+019r89y3v6Hm8J/aYdulQ7YUlXP99+7O8BSS71JPkf39UZSWKszi3f1iA7/rs8PcnbuvNj2Rh7viIEzxeD9P67AqzQDHnHyiRA7wYWl3Nu/9vDUTFxBz3GY98syO4/08rcOrOg0cXV3qP/+59/OrV/c27+QkQwkRKIJfPOZPJXsflbYhXmsS44kTd9I5bJCSoaY8AAXX1qJUJ0QFiDO6J0Dl8Yb0tOfmcrlULudqaV0R7v8IRX4xVAsW0n3PimklDWib29Gfsm2hCjIPADHv5aFDL0T5YkuVJk20L0WAaE5HHbqtemat/UAi0xyMTHE/FefaWiQyUT0q6RxExUnTRGEoETKt7aEI4kFVmp9CAZQIGKLZejSEPpyqxZGpGcsyqSwLEA5FKc57apw+La7SExGMFClSpEiRIkWKFCnSEVBpvFb7nKg2Xo1qY4dUYbxy/+27L/oDOvHFL2/vVyKcB1Lt/i8nCLQQUZq/vP08Qtleld9bEPTTPPE27JnGkZje7YtQkIwKjy01flCKwPFtpy/2I9b9/Xs0M8Z30VRCG42/PbGfc8Rh5t3nnb7Qj161+//37gRLcZr44azn188jSzyYStXP79//9Zd3v3/xxW/9J/p/++K3L37/5e2v9yu10v65zv8DvBgupHjx7McAAAAASUVORK5CYII="

In [14]:
base_64_string = "..."

In [213]:
len(base_64_string)

8976

In [ ]:
import base64
import re

def is_base64(string):
    base64_pattern = re.compile(r'^[A-Za-z0-9+/=\s]+$')
    return base64_pattern.fullmatch(string) is not None

def extract_text_from_base64(base64_data, output_file_path=None):
    try:

        if not is_base64(base64_data.strip()):
            print("Error: The file does not contain valid Base64-encoded text.")
            return None

        decoded_data = base64.b64decode(base64_data)

        try:
            decoded_text = decoded_data.decode('utf-16')
            if output_file_path:
                with open(output_file_path, 'w') as output_file:
                    output_file.write(decoded_text)
            return decoded_text
        except UnicodeDecodeError:
            if output_file_path:
                with open(output_file_path, 'wb') as output_file:
                    output_file.write(decoded_data)
            return None

    except (base64.binascii.Error, ValueError) as e:
        print(f"Error: Failed to decode Base64 content. Details: {e}")

text = extract_text_from_base64(base_64_string, "base_64_string_decode.docx")
if text:
    print("Decoded text:")
    print(text) 

In [61]:
len(base_64_string)

74188

In [267]:
import requests

url = "http://localhost:8000/predict"

# question = "major roads in abudhabi"
# question = "Can you give me the details about M001 milestone"
question = "Can you give me details about active projects?"

response = requests.post(url, json={"question": question})

if response.status_code == 200:
    result = response.json()
    print("Response Status:", result['status'])
    print("Question:", result['question'])
    print("Response:", result['response'])
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Response Status: 200
Question: Can you give me details about active projects?
Response: Based on the provided data, there are currently 4 active projects in Abu Dhabi. Here's a summary of the projects:

- **Project Alpha**: An infrastructure project with a completion rate of 85.50% and a planned budget of $1,500,000. It started on January 15, 2023, and is expected to be completed on February 15, 2023.
- **Project Beta**: A healthcare project with a completion rate of 100.00% and a planned budget of $2,000,000. It started on February 1, 2023, and was completed on August 1, 2023.
- **Project Epsilon**: An energy project with a completion rate of 60.00% and a planned budget of $2,500,000. It started on May 1, 2023, and is expected to be completed on June 1, 2023.

These projects are currently in progress, and their status is being monitored.


In [250]:
print(result['response'])

The M001 milestone, 'Initiation Phase', has been completed. It started on January 15, 2023, and was finished on February 15, 2023, taking 31 days to complete. The project is on track with a completion rate of 85.50% and has a planned budget of $1,500,000, with an actual budget of $1,450,000.


In [ ]:
doc_text_content = {
    "Attachment1.pdf": "5Law No. (20) of 2023 co"
}



5Law No. (20) of 2023 co
